In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

# ---------------------------------------------------------
# 1. Load dataset (same as practical)
# ---------------------------------------------------------
file_path = "EURUSD_tick_OK-2.csv"     # adjust path if needed
df = pd.read_csv(file_path)

print("Data shape:", df.shape)
print("Columns:", df.columns.tolist())
# ['Vol_Ask_N', 'Ask_N_200_3', 'DateDelta1_N', 'Bid_N_200_3', 'Vol_Bid_N']

# ---------------------------------------------------------
# 2. Create 50-tick windows (same logic as practical)
#    X: (samples, 50, 5)   Y: (samples, 2) -> next tick Bid & Ask
# ---------------------------------------------------------
N = 50           # number of ticks in each window (given in practical)
n_small = 250000 # subset of ticks used in practical (≈ 1 week)

data = df.iloc[:n_small].values   # shape: (n_small, 5)

BID_COL = df.columns.get_loc("Bid_N_200_3")
ASK_COL = df.columns.get_loc("Ask_N_200_3")

X_list = []
y_list = []

for i in range(n_small - N - 1):
    # 50-tick window of all 5 normalised features
    X_list.append(data[i:i + N, :])
    # target = Bid & Ask at the next tick after the window
    y_list.append(data[i + N, [BID_COL, ASK_COL]])

X = np.array(X_list)   # (samples, 50, 5)
y = np.array(y_list)   # (samples, 2)

print("X shape:", X.shape)
print("y shape:", y.shape)

# ---------------------------------------------------------
# 3. Train–test split (same dataset as practical, no shuffle)
# ---------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# ---------------------------------------------------------
# 4. Epochs from your SID (XXXXXXZY = 2421463  → Z=6, Y=3)
# ---------------------------------------------------------
sid = "2421463"
Z = int(sid[-2])   # 6
Y = int(sid[-1])   # 3

if Z != 0 and Y != 0:
    EPOCHS = Z + Y           # Z + Y
elif Z == 0 and Y != 0:
    EPOCHS = 10 + Y
else:
    EPOCHS = 10

print("Using epochs =", EPOCHS)     # should print 9

BATCH_SIZE = 50                    # requirement: batch_size = 50

# ---------------------------------------------------------
# 5. Create CNN (same as practical, but kernel_size/core = 5)
# ---------------------------------------------------------
model = keras.Sequential([

    # Practical: Conv1D(50, 9, ...)  → Week-5: core size 5
    keras.layers.Conv1D(
        filters=50,
        kernel_size=5,                     # ← changed from 9 to 5
        padding='same',
        input_shape=(50, 5),
        activation=tf.nn.relu,
        kernel_initializer="normal"
    ),

    keras.layers.MaxPooling1D(7),

    # Practical: Conv1D(100, 7, ...) → Week-5: core size 5
    keras.layers.Conv1D(
        filters=100,
        kernel_size=5,                     # ← changed from 7 to 5
        padding='same',
        activation=tf.nn.relu,
        kernel_initializer="normal"
    ),

    keras.layers.GlobalMaxPooling1D(),

    keras.layers.Dense(25, activation=tf.nn.relu, kernel_initializer="normal"),

    # Output: Bid and Ask prices at next tick (2 neurons)
    keras.layers.Dense(2)
])

# ---------------------------------------------------------
# 6. Compile model (same style as practical; MAE for requirement)
# ---------------------------------------------------------
model.compile(
    optimizer="adam",
    loss="mae",          # if your practical used mse, you can switch back to 'mse'
    metrics=["mae"]
)

# Show architecture for Lab Logbook screenshot
print(model.summary())

# ---------------------------------------------------------
# 7. Train CNN  (changed batch_size + epochs as required)
# ---------------------------------------------------------
history = model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,          # ← 9 for your SID
    batch_size=BATCH_SIZE,  # ← 50
    validation_split=0.1,
    verbose=1
)

# ---------------------------------------------------------
# 8. Evaluate test MAE (for comparison with practical CNN)
# ---------------------------------------------------------
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print("Test MAE:", test_mae)


Data shape: (1100000, 5)
Columns: ['Vol_Ask_N', 'Ask_N_200_3', 'DateDelta1_N', 'Bid_N_200_3', 'Vol_Bid_N']
X shape: (249949, 50, 5)
y shape: (249949, 2)
Using epochs = 9


C:\Users\sudha\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 50, 50)              │           1,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 7, 50)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 7, 100)              │          25,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 100)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 25)                  │           2,525 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              52 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 28,977 (113.19 KB)

 Trainable params: 28,977 (113.19 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - loss: 0.0244 - mae: 0.0244 - val_loss: 0.0201 - val_mae: 0.0201
Epoch 2/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - loss: 0.0180 - mae: 0.0180 - val_loss: 0.0202 - val_mae: 0.0202
Epoch 3/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - loss: 0.0177 - mae: 0.0177 - val_loss: 0.0205 - val_mae: 0.0205
Epoch 4/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - loss: 0.0175 - mae: 0.0175 - val_loss: 0.0197 - val_mae: 0.0197
Epoch 5/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - loss: 0.0175 - mae: 0.0175 - val_loss: 0.0195 - val_mae: 0.0195
Epoch 6/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 34s 9ms/step - loss: 0.0174 - mae: 0.0174 - val_loss: 0.0199 - val_mae: 0.0199
Epoch 7/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 36s 10ms/step - loss: 0.0173 - mae: 0.0173 - val_loss: 0.0206 - val_mae: 0.0206
Epoch 8/9
3600/3600 ━━━━━━━━━━━━━━━━━━━━ 32s 9ms/step - loss: 0.0173 - mae: 0.0173 - val_loss: 0.0194 - val_mae: 0.0194
Epoch 9/9
3600/3600 ━━━━━━━━━━